<a href="https://colab.research.google.com/github/q890003/ML_logistic-regression-EM/blob/master/ML_hw4_EM_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import dill
import numpy as np
import math
from itertools import product 
from struct import unpack 
import numba as nb

def get_img(img_file_path):
    with open(img_file_path, "rb") as img_file:
        magic, num_imgs, rows, cols = unpack(">IIII", img_file.read(16))

        imgs_mtx = np.zeros((num_imgs, rows, cols), dtype=int)
        for img in range(num_imgs):
            for (row, col) in product(range(rows), range(cols)):
                tmp = unpack(">B", img_file.read(1))
                tmp = int(str(tmp[0]))
                imgs_mtx[img][row,col] = tmp
    return imgs_mtx, num_imgs

def get_label(lbl_file_path):
    with open(lbl_file_path, "rb") as label_file:
        magic, num_itms = unpack(">II", label_file.read(8))
        lbls_mtx = np.zeros((num_itms), dtype=int)
        for img in range(num_itms):
            label = unpack(">B", label_file.read(1))
            label = int(str(label[0]))
            lbls_mtx[img] = label
    return lbls_mtx

label_types = 10
gray_bins = 2
rows = 28
cols = 28

train_img_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw4/train-images.idx3-ubyte"
train_label_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw4/train-labels.idx1-ubyte"
train_img_mtx, train_num_imgs = get_img(train_img_pth)
train_label_mtx = get_label(train_label_pth)

labels_count = np.zeros((label_types), dtype=float)
for img in range(train_num_imgs):
    lbl = train_label_mtx[img]
    labels_count[lbl] += 1
    train_img_mtx[img] = train_img_mtx[img] // 128
  

In [25]:
@nb.jit
def estep(lamda, mu, train_img_mtx):
    weight = np.zeros((60000, 10), dtype=np.float64)
    for img in range(60000):
        for label in range(10):
            weight[img][label] = np.prod(2* (mu[label]*train_img_mtx[img] + (1- mu[label])*(1 - train_img_mtx[img]) ))
        weight[img] = weight[img]*lamda
        total = np.sum(weight[img])
        weight[img] /= total
    return weight

@nb.jit
def mstep(weight, train_img_mtx):
    mu_new = np.zeros((10,28,28), dtype=np.float64)

    for img in range(60000):
        w_ = weight[img]
        w_[ w_ == 0] = 0.001
        weight[img] = w_

    for label in range(10):
        for img in range(60000):
            mu_new[label] += weight[img][label] * train_img_mtx[img]  
    
    sigma_w = np.sum(weight, axis=0) 
    
    for label in range(10): 
        mu_new[label] /= sigma_w[label] 
    #recalculate lamda of each cluster. 
    lamda = sigma_w/60000
    return mu_new, lamda

@nb.jit
def map_cluster2number(train_label_mtx, weight, mu):
    count = np.zeros((10, 10), dtype=np.float64)
    map_cltr2num = np.zeros((10), dtype=np.int32)
    idx = 0
    for img in range(60000):
        count[train_label_mtx[img], np.argmax(weight[img]) ] += 1
    for label in range(10):
        idx = np.argmax(count[label])
        map_cltr2num[label] = idx
        for i in range(10):
            count[i,idx] = -1
    return map_cltr2num

def print_imag(map_cltr2num, mu, mu_new, iteration):
    imagination_number = mu_new.copy()
    imagination_number[imagination_number > 0.5] = 1
    imagination_number[imagination_number <= 0.5] = 0
    print("------------------------------------------------------------")
    for label in range(10):
        print("labeled class ",label,":")
        for r in range(28):
            for c in range(28):
                pxl = int(imagination_number[map_cltr2num[label]][r][c])
                print(pxl ,end=" ")
            print("")
    print("No. of Iteration: ", iteration," Difference: ",  np.sum(abs(mu_new - mu)),"\n")
    print("------------------------------------------------------------")

# init
lamda = np.full((label_types), 1/10)  #prob of a nmber over numbers 0~9
mu = np.random.rand(label_types,28,28)  #each cluster has 28*28 dim mu.
weight = np.zeros((train_num_imgs, label_types), dtype=float)  #record weights of each data to 10 clusters.
converge_count = 0
mu_change = 99999
#EM algorithm
while True:
    # Expectation step:
    weight = estep(lamda, mu, train_img_mtx)

    # Maximization step:   recalculate mu to each image and lamda of each cluster. 
    mu_new, lamda = mstep(weight, train_img_mtx)
    
    if converge_count % 10 == 0:
        #mapping clusters to labels, 0~9
        map_cltr2num = map_cluster2number(train_label_mtx, weight, mu_new)

        #print imagination number
        print_imag(map_cltr2num, mu,mu_new, converge_count)
    
    mu_change_new = np.sum(abs(mu_new - mu))
    if mu_change_new < 0.1 or mu_change_new > mu_change*2 :
        # print(mu_change_new, "vs", mu_change)
        mu = mu_new
        break

    #update
    mu = mu_new
    mu_change = mu_change_new
    converge_count += 1


------------------------------------------------------------
labeled class  0 :
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 

In [26]:
def confusionMtrix( mu, map_cltr2num, train_img_mtx, train_label_mtx):
    CM = np.zeros((10,4), dtype = np.int32)   #TP, FN, FP, TN
    for label in range(10):
        for img in range(60000):
            true_label = train_label_mtx[img]
            cluster = np.argmax(weight[img])
            prediction = np.where(map_cltr2num == cluster)[0].item()
        
            if label  == true_label:
                if label == prediction:
                    CM[label][0] += 1
                else:
                    CM[label][2] += 1
            else: 
                if label == prediction:
                    CM[label][1] += 1
                else:
                    CM[label][3] += 1
    return CM

tp = 0 #true positive
CM = confusionMtrix(mu, map_cltr2num, train_img_mtx, train_label_mtx)
for k in range(10):
    tp += CM[k][0]
    Sensitivity = CM[k][0]/ (CM[k][0]+CM[k][1])
    Specificity = CM[k][3]/ (CM[k][2]+CM[k][3])
    print("")
    print("Confusion Matrix",k,":")
    print("\t\t","Predict number",k,"\tPredict not number",k)
    print("Is number",k,"\t\t",CM[k][0],"\t\t\t",CM[k][1])
    print("Isn't number",k,"\t\t",CM[k][2],"\t\t\t",CM[k][3])
    print("")
    print("Sensitivity (Successfully predict number ",k,")     :",Sensitivity)
    print("Specificity (Successfully predict not number ",k,") :",Specificity)
    print("")
    print("-----------------------------------------------------------")
print("Total iteration to converge: ",converge_count)
print("Total error rate: ",  (60000- tp) / 60000)


Confusion Matrix 0 :
		 Predict number 0 	Predict not number 0
Is number 0 		 4355 			 302
Isn't number 0 		 1568 			 53775

Sensitivity (Successfully predict number  0 )     : 0.9351513850118102
Specificity (Successfully predict not number  0 ) : 0.9716676002385125

-----------------------------------------------------------

Confusion Matrix 1 :
		 Predict number 1 	Predict not number 1
Is number 1 		 3676 			 1072
Isn't number 1 		 3066 			 52186

Sensitivity (Successfully predict number  1 )     : 0.7742207245155855
Specificity (Successfully predict not number  1 ) : 0.944508796061681

-----------------------------------------------------------

Confusion Matrix 2 :
		 Predict number 2 	Predict not number 2
Is number 2 		 4451 			 460
Isn't number 2 		 1507 			 53582

Sensitivity (Successfully predict number  2 )     : 0.9063327224597841
Specificity (Successfully predict not number  2 ) : 0.9726442665504911

-----------------------------------------------------------

Confusion Ma